In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/ant-1.7.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,ant,1.7,org.apache.tools.ant.taskdefs.rmic.RmicAdapter...,3,1,0,10,18,3,1,...,0.000000,0,0.000000,0.444444,0,0,32.666667,1,0.6667,0
1,ant,1.7,org.apache.tools.ant.taskdefs.optional.perforc...,5,2,0,4,13,0,1,...,1.000000,1,0.700000,0.500000,0,0,13.400000,1,0.6000,0
2,ant,1.7,org.apache.tools.ant.taskdefs.optional.junit.O...,1,2,0,1,3,0,0,...,0.000000,0,1.000000,1.000000,0,0,6.000000,0,0.0000,0
3,ant,1.7,org.apache.tools.ant.taskdefs.optional.perforc...,8,1,9,13,20,12,9,...,0.200000,1,0.000000,0.406250,0,0,11.000000,1,0.8750,0
4,ant,1.7,org.apache.tools.ant.taskdefs.WaitFor,9,3,0,5,26,16,0,...,1.000000,0,0.800000,0.388889,0,0,19.000000,2,1.0000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
740,ant,1.7,org.apache.tools.ant.taskdefs.Javadoc,92,3,0,34,261,3726,8,...,0.970588,11,0.291339,0.112476,2,2,28.021739,15,1.5543,4
741,ant,1.7,org.apache.tools.ant.types.selectors.BaseSelector,6,3,6,10,10,3,7,...,1.000000,0,0.857143,0.500000,0,0,6.500000,3,1.5000,0
742,ant,1.7,org.apache.tools.ant.types.resources.selectors...,7,3,5,9,26,0,5,...,1.000000,0,0.857143,0.314286,1,3,19.000000,3,1.5714,0
743,ant,1.7,org.apache.tools.ant.taskdefs.compilers.Gcj,5,2,0,8,34,8,1,...,1.000000,0,0.884615,1.000000,0,0,42.000000,11,3.4000,1


In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
155,6,5,2,8,21,15,2,6,5,2.000000,83,0.000000,0,0.955357,0.416667,1,1,12.833333,3,1.1667
407,2,3,0,3,7,1,0,3,2,2.000000,25,0.000000,0,0.969697,1.000000,0,0,11.500000,1,0.5000
8,9,1,0,5,19,8,4,1,9,0.531250,119,1.000000,0,0.000000,0.518519,0,0,11.777778,3,1.4444
433,28,3,0,18,132,130,3,15,19,0.846561,1139,0.928571,2,0.596774,0.151515,3,5,39.178571,6,1.6786
657,33,1,6,22,135,316,7,15,10,0.940625,1367,0.600000,5,0.000000,0.176309,0,0,39.818182,10,2.0909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,21,1,2,14,47,124,10,4,18,0.700000,575,0.857143,0,0.000000,0.325397,0,0,26.047619,10,2.6190
386,8,3,0,5,25,16,1,5,7,0.761905,152,1.000000,0,0.840909,0.750000,0,0,17.625000,3,1.3750
319,14,3,0,7,46,27,0,7,8,0.837607,372,0.666667,0,0.740000,0.214286,1,1,24.928571,5,1.8571
303,5,2,2,9,10,6,7,2,4,0.900000,185,0.400000,6,0.800000,0.500000,1,1,34.000000,5,1.2000


In [7]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
81,4,1,0,8,20,0,4,4,4,0.777778,106,1.000000,1,0.000000,0.625000,0,0,24.750000,5,1.7500
353,21,1,0,2,36,38,2,0,17,0.631818,620,0.818182,0,0.000000,0.293651,0,0,28.000000,3,0.8571
51,17,5,0,9,34,118,2,7,17,0.975000,137,0.500000,1,0.905882,0.270588,3,5,6.470588,2,1.0588
677,9,3,0,3,20,34,1,2,9,0.625000,177,1.000000,1,0.761905,0.333333,1,3,18.555556,4,1.1111
660,30,1,0,10,104,429,2,9,5,1.013793,1038,0.800000,0,0.000000,0.108333,0,0,33.266667,9,1.9000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,38,3,4,16,68,601,4,12,13,0.928458,534,1.000000,1,0.500000,0.289474,0,0,12.605263,5,1.3684
486,3,2,1,4,5,3,1,3,3,2.000000,11,0.000000,0,0.818182,0.666667,0,0,2.666667,1,0.6667
548,2,1,0,4,2,1,3,2,2,2.000000,2,0.000000,0,0.000000,0.666667,0,0,0.000000,1,1.0000
197,2,2,0,2,15,1,0,2,2,2.000000,73,0.000000,0,0.875000,0.750000,0,0,35.500000,6,3.0000


In [8]:
y_train = train['bug']
y_test = test['bug']

In [9]:
y_train

155    1
407    0
8      0
433    3
657    0
      ..
167    0
386    0
319    0
303    0
658    2
Name: bug, Length: 596, dtype: int64

In [10]:
y_test

81     0
353    1
51     1
677    0
660    3
      ..
470    0
486    0
548    0
197    0
403    0
Name: bug, Length: 149, dtype: int64

In [11]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [12]:
x_train_np

array([[ 6.        ,  5.        ,  2.        , ..., 12.83333333,
         3.        ,  1.1667    ],
       [ 2.        ,  3.        ,  0.        , ..., 11.5       ,
         1.        ,  0.5       ],
       [ 9.        ,  1.        ,  0.        , ..., 11.77777778,
         3.        ,  1.4444    ],
       ...,
       [14.        ,  3.        ,  0.        , ..., 24.92857143,
         5.        ,  1.8571    ],
       [ 5.        ,  2.        ,  2.        , ..., 34.        ,
         5.        ,  1.2       ],
       [ 2.        ,  2.        ,  0.        , ..., 78.5       ,
         1.        ,  0.5       ]])

In [13]:
x_test_np

array([[ 4.        ,  1.        ,  0.        , ..., 24.75      ,
         5.        ,  1.75      ],
       [21.        ,  1.        ,  0.        , ..., 28.        ,
         3.        ,  0.8571    ],
       [17.        ,  5.        ,  0.        , ...,  6.47058824,
         2.        ,  1.0588    ],
       ...,
       [ 2.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [ 2.        ,  2.        ,  0.        , ..., 35.5       ,
         6.        ,  3.        ],
       [ 1.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [14]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [15]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [16]:
train_x

array([[[ 6.        ,  5.        ,  2.        , ..., 12.83333333,
          3.        ,  1.1667    ]],

       [[ 2.        ,  3.        ,  0.        , ..., 11.5       ,
          1.        ,  0.5       ]],

       [[ 9.        ,  1.        ,  0.        , ..., 11.77777778,
          3.        ,  1.4444    ]],

       ...,

       [[14.        ,  3.        ,  0.        , ..., 24.92857143,
          5.        ,  1.8571    ]],

       [[ 5.        ,  2.        ,  2.        , ..., 34.        ,
          5.        ,  1.2       ]],

       [[ 2.        ,  2.        ,  0.        , ..., 78.5       ,
          1.        ,  0.5       ]]])

In [17]:
test_y

array([0, 1, 1, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0,
       2, 1, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [18]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0001) , metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [19]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 200, callbacks=[tensorboard_callback])

Epoch 1/200
19/19 [==============================] - 4s 14ms/step - loss: 1.7291 - mse: 1.7291 - mae: 0.5014 - root_mean_squared_error: 1.3149 - mean_squared_logarithmic_error: 0.2854
Epoch 2/200
19/19 [==============================] - 0s 5ms/step - loss: 1.7073 - mse: 1.7073 - mae: 0.5105 - root_mean_squared_error: 1.3066 - mean_squared_logarithmic_error: 0.2759
Epoch 3/200
19/19 [==============================] - 0s 5ms/step - loss: 1.6844 - mse: 1.6844 - mae: 0.5223 - root_mean_squared_error: 1.2979 - mean_squared_logarithmic_error: 0.2662
Epoch 4/200
19/19 [==============================] - 0s 5ms/step - loss: 1.6526 - mse: 1.6526 - mae: 0.5368 - root_mean_squared_error: 1.2855 - mean_squared_logarithmic_error: 0.2545
Epoch 5/200
19/19 [==============================] - 0s 5ms/step - loss: 1.6110 - mse: 1.6110 - mae: 0.5572 - root_mean_squared_error: 1.2692 - mean_squared_logarithmic_error: 0.2413
Epoch 6/200
19/19 [==============================] - 0s 5ms/step - loss: 1.5537 - ms

19/19 [==============================] - 0s 5ms/step - loss: 0.5767 - mse: 0.5767 - mae: 0.3658 - root_mean_squared_error: 0.7594 - mean_squared_logarithmic_error: 0.1058
Epoch 89/200
19/19 [==============================] - 0s 5ms/step - loss: 0.5353 - mse: 0.5353 - mae: 0.3463 - root_mean_squared_error: 0.7317 - mean_squared_logarithmic_error: 0.0930
Epoch 90/200
19/19 [==============================] - 0s 7ms/step - loss: 0.5519 - mse: 0.5519 - mae: 0.3670 - root_mean_squared_error: 0.7429 - mean_squared_logarithmic_error: 0.1057
Epoch 91/200
19/19 [==============================] - 0s 5ms/step - loss: 0.5288 - mse: 0.5288 - mae: 0.3596 - root_mean_squared_error: 0.7272 - mean_squared_logarithmic_error: 0.1021
Epoch 92/200
19/19 [==============================] - 0s 5ms/step - loss: 0.5699 - mse: 0.5699 - mae: 0.3285 - root_mean_squared_error: 0.7549 - mean_squared_logarithmic_error: 0.0939
Epoch 93/200
19/19 [==============================] - 0s 5ms/step - loss: 0.6662 - mse: 0.666

19/19 [==============================] - 0s 5ms/step - loss: 0.2400 - mse: 0.2400 - mae: 0.2083 - root_mean_squared_error: 0.4899 - mean_squared_logarithmic_error: 0.0513
Epoch 177/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2438 - mse: 0.2438 - mae: 0.2220 - root_mean_squared_error: 0.4938 - mean_squared_logarithmic_error: 0.0536
Epoch 178/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2593 - mse: 0.2593 - mae: 0.2213 - root_mean_squared_error: 0.5092 - mean_squared_logarithmic_error: 0.0540
Epoch 179/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2546 - mse: 0.2546 - mae: 0.2194 - root_mean_squared_error: 0.5046 - mean_squared_logarithmic_error: 0.0532
Epoch 180/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2621 - mse: 0.2621 - mae: 0.2304 - root_mean_squared_error: 0.5120 - mean_squared_logarithmic_error: 0.0594
Epoch 181/200
19/19 [==============================] - 0s 5ms/step - loss: 0.2569 - mse: 

In [20]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:26:35 ago. (Use '!kill 3750' to kill it.)

In [21]:
score = model.evaluate(test_x, test_y)
score

5/5 [==============================] - 1s 2ms/step - loss: 0.4196 - mse: 0.4196 - mae: 0.3052 - root_mean_squared_error: 0.6477 - mean_squared_logarithmic_error: 0.1028


[0.41957366466522217,
 0.41957366466522217,
 0.3052019774913788,
 0.6477450728416443,
 0.10277536511421204]